### Урок 4. Парсинг HTML. XPath

* Выберите веб-сайт с табличными данными, который вас интересует.
* Напишите код Python, использующий библиотеку requests для отправки HTTP GET-запроса на сайт и получения HTML-содержимого страницы.
* Выполните парсинг содержимого HTML с помощью библиотеки lxml, чтобы извлечь данные из таблицы.
* Сохраните извлеченные данные в CSV-файл с помощью модуля csv.

Ваш код должен включать следующее:

- Строку агента пользователя в заголовке HTTP-запроса, чтобы имитировать веб-браузер и избежать блокировки сервером.
- Выражения XPath для выбора элементов данных таблицы и извлечения их содержимого.
- Обработка ошибок для случаев, когда данные не имеют ожидаемого формата.
- Комментарии для объяснения цели и логики кода.

Целью данной работы будет попытка получить данные по турнирам настольного тенниса от Лига-Про.

1. Пытаемся получить ответ с get-запроса по интересующему нас дню
2. Получить информацию по этому дню о предстоящих турнирах со списком участников
3. Сформировать из полученных данных датафрейм
4. Сохранить данных в табличном формате в файле csv

In [33]:
#!pip install lxml

In [34]:
# импортируем нужные библиотеки
from lxml import html
import requests
from pprint import pprint
import csv
import pandas as pd

In [35]:
url='https://tt.sport-liga.pro/'

# задаем user-agent
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
    }
# в params надо указать интересующую нас дату в формате YYYY:MM:DD
# param = {
#     'year': '2024',
#     'month': '05',
#     'day': '7'
# }

url_tour = 'tours/'
session = requests.Session()

In [36]:
# просьба ввести данные пользователя из консоли (В РАЗРАБОТКЕ)
date = input('Введите дату в формате DD:MM:YYYY ')
day, month, year = date.split('.')
params = {
    'year': year,
    'month': month,
    'day': day
}

In [37]:
# делаем get-запрос
response = session.get(url+url_tour, params=params, headers=header)
# проверяем статус-код
if response.status_code < 400:
    # если да, печатаем статус код и представляем ответ в виде дерева html
    print(response.status_code)
    tree = html.fromstring(response.text)
else:
    print('Что-то пошло не так')

200


Необходимые данные находятся на странице в нижней таблице. Также для сбора фамилий участников турнира нам нужно переходить на его страницу (ее можно "достать" из таблицы.)

In [38]:
# находим нужную таблицу на странице
table = tree.xpath('//table[@class="table"]//tr')
# находим ссылки на переход к просмотру турнира
ref = tree.xpath('.//td[@class="tournament-name"]/a/@href')

Для объединения всех данных в один список будем проходиться циклом for сразу по двум спискам, для этого объединим их функцией zip

In [39]:
tournaments = []
for row, item in zip(table[1:], ref):
    # сделаем get-запрос на страницу турнира
    res = session.get(url+item, headers=header)
    # и получим дерево из его ответа
    tour = html.fromstring(res.text)
    # создадим словарик для каждого турнира
    tours_info = {}
    # из даты нам нужно только время начало турнира
    time = row.xpath('.//td[@class="tournament-date"]/text()')
    time = [el.split()[-1] for el in time][0]

    # из названия турнира мы возьмем
    tours = row.xpath('.//td[@class="tournament-name"]/a/text()')
    # название стола
    tables = [el.split()[1][:-1] for el in tours][0]
    # уровень лиги
    liga = [el.split()[3] for el in tours][0]

    # участников турнира получим из страницы турнира
    parties = tour.xpath('//td[@class="nowrap"]/a/text()')

    tours_info['date'] = date
    tours_info['time'] = time
    tours_info['tables'] = tables
    tours_info['liga'] = liga
    tours_info['parties'] = parties
    
    # добавим каждый словарь в список
    tournaments.append(tours_info)


In [40]:
pprint(tournaments)

[{'date': '15.5.24',
  'liga': '500-550',
  'parties': ['Кутузов О', 'Илюхин Е', 'Игнатьев Д', 'Иванов А-р', 'Пандур И'],
  'tables': 'А6',
  'time': '23:45'},
 {'date': '15.5.24',
  'liga': '400-450',
  'parties': ['Лбов Ю', 'Федоров А', 'Руденко В', 'Сорокин М', 'Михайлов И'],
  'tables': 'А4',
  'time': '23:45'},
 {'date': '15.5.24',
  'liga': '450-500',
  'parties': ['Попов О', 'Зюганов О', 'Тарасенков Н', 'Мамека М', 'Анисимов Е'],
  'tables': 'А5',
  'time': '23:30'},
 {'date': '15.5.24',
  'liga': '900-1000',
  'parties': ['Салкин Р', 'Фролов А И', 'Лукьянов П', 'Дедов В'],
  'tables': 'А9',
  'time': '20:00'},
 {'date': '15.5.24',
  'liga': '350-400',
  'parties': ['Самоха Е', 'Карюхин А', 'Мельников М', 'Арлашов А'],
  'tables': 'А3',
  'time': '20:00'},
 {'date': '15.5.24',
  'liga': '500-550.',
  'parties': ['Шафиев Р', 'Атоманчик К', 'Ляльков В', 'Дехтяренко А'],
  'tables': 'А15',
  'time': '20:00'},
 {'date': '15.5.24',
  'liga': '800-900',
  'parties': ['Хоменко Р', 'Евл

Запишем полученную информацию в csv-файл

In [41]:
# with open('./tournaments.csv', 'w', newline='') as f:
#     writer = csv.DictWriter(f, fieldnames=tournaments[0].keys())
#     writer.writeheader
#     for row in tournaments:
#         writer.writerow(row)
# print('Данные успешно записаны')

Для наглядности представим наш список в виде датафрейма

In [42]:
df = pd.DataFrame(tournaments)
df

,date,time,tables,liga,parties
0,15.5.24,23:45,А6,500-550,"[Кутузов О, Илюхин Е, Игнатьев Д, Иванов А-р, ..."
1,15.5.24,23:45,А4,400-450,"[Лбов Ю, Федоров А, Руденко В, Сорокин М, Миха..."
2,15.5.24,23:30,А5,450-500,"[Попов О, Зюганов О, Тарасенков Н, Мамека М, А..."
3,15.5.24,20:00,А9,900-1000,"[Салкин Р, Фролов А И, Лукьянов П, Дедов В]"
4,15.5.24,20:00,А3,350-400,"[Самоха Е, Карюхин А, Мельников М, Арлашов А]"
5,15.5.24,20:00,А15,500-550.,"[Шафиев Р, Атоманчик К, Ляльков В, Дехтяренко А]"
6,15.5.24,19:45,А6,800-900,"[Хоменко Р, Евлампиев В, Денисов А-м, Амбросим..."
7,15.5.24,19:45,А4,600-700,"[Афанасьев М А, Ляличев Д, Шкурко А, Гребенюк Б]"
8,15.5.24,19:30,А5,300-350,"[Невежин Г, Яковлев А, Нурмухамедов М, Савченк..."
9,15.5.24,16:00,А3,400-450,"[Соколов А, Алексеев А, Гусев М, Савинский А]"


Вывод: В результате проделанной работы мы получаем датафрейм, содержащий нужную информацию. Этот датафрейм планируется использовать для подбора турнира с более предпочтительными игроками.

Также планируется исследовать каждый турнир на предмет статистики личных встреч между игроками

In [43]:
#df = df[df['tables'] != 'А15']
df = df[df['time'] < '23:00'] 
df = df[df['time'] > '15:00']
df

,date,time,tables,liga,parties
3,15.5.24,20:00,А9,900-1000,"[Салкин Р, Фролов А И, Лукьянов П, Дедов В]"
4,15.5.24,20:00,А3,350-400,"[Самоха Е, Карюхин А, Мельников М, Арлашов А]"
5,15.5.24,20:00,А15,500-550.,"[Шафиев Р, Атоманчик К, Ляльков В, Дехтяренко А]"
6,15.5.24,19:45,А6,800-900,"[Хоменко Р, Евлампиев В, Денисов А-м, Амбросим..."
7,15.5.24,19:45,А4,600-700,"[Афанасьев М А, Ляличев Д, Шкурко А, Гребенюк Б]"
8,15.5.24,19:30,А5,300-350,"[Невежин Г, Яковлев А, Нурмухамедов М, Савченк..."
9,15.5.24,16:00,А3,400-450,"[Соколов А, Алексеев А, Гусев М, Савинский А]"
10,15.5.24,16:00,А15,600-700.,"[Кошель В, Король Д, Логунов А, Мурашко В]"
11,15.5.24,15:45,А4,450-500,"[Фролов А Г, Алов Р, Молодцов Д, Пузырев С]"
12,15.5.24,15:45,А6,250-300,"[Смолин Е, Константинов С, Попроцкий А, Пряхин В]"
